In [1]:
import torch
import torch.nn as nn
from torchvision import models
from train import  read_data
import pandas as pd
from torch.optim.lr_scheduler import StepLR
from kaggleImageFolder import KaggleImageFolder

In [2]:
batch_size = 512
labels_csv = './labels.csv'
root = 'D:/DATA/dog bread'


In [3]:
train_features = torch.load(root + '/train_features.pth')
valid_features = torch.load(root + '/valid_features.pth')
test_features = torch.load(root + '/test_features.pth')

In [4]:
def accuracy(model, dataloader, n_data):
    correct = 0
    model.eval()
    for x, y in dataloader:
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
        z = model(x)
        yhat = torch.argmax(z, 1)
        correct += (y == yhat).sum().item()

    acc = correct / n_data
    return acc

In [5]:
class Classifier(nn.Module):
    def __init__(self, input_dim, n_classes):
        super(Classifier, self).__init__()
        self.arch = self.__class__.__name__
        self.n_classes = n_classes
        
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.Dropout(p=0.5),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.Dropout(p=0.5),
            nn.ReLU(inplace=True),
            nn.Linear(512, n_classes)
        )
        torch.nn.init.kaiming_uniform_(self.model[0].weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.model[3].weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.model[-1].weight, nonlinearity='relu')
        
    def forward(self, x):
        x = self.model(x)
        return x
    
    def save_model(self, save_path):
        checkpoint = {
            'model_state_dict': self.state_dict(),
            'n_classes': self.n_classes
        }
        torch.save(checkpoint, save_path)

In [6]:
def train(model, train_dataloader, valid_dataloader, n_train, n_val, criterion, optimizer, epochs=20, scheduler=None, save_path='./'):
    train_loss_list = []
    train_acc_list = []
    valid_acc_list = []

    it = 1

    for epoch in range(epochs):
        if scheduler is not None:
            scheduler.step()

        epoch_losses = []
        model.train()
        for i, (x, y) in enumerate(train_dataloader):
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            epoch_losses.append(loss.data.item())
            loss.backward()
            optimizer.step()

            it += 1

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        mean_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        print("Epoch: {}/{}...".format(epoch + 1, epochs),
              "Train Loss: {:.4f}".format(mean_epoch_loss))
        train_loss_list.append(mean_epoch_loss)

        train_acc_list.append(accuracy(model, train_dataloader, n_train))

        if valid_dataloader is not None:
            acc = accuracy(model, valid_dataloader, n_val)
            if len(valid_acc_list) > 0 and acc > max(valid_acc_list):
                model.save_model(save_path + '/{}-checkpoint-best.pth'.format(model.arch))
                optimizer_params = {"optimizer_params": optimizer.state_dict()}
                torch.save(optimizer_params, save_path + "/{}-{}-checkpoint-best.pth".format(
                    model.arch, optimizer.__class__.__name__))
            valid_acc_list.append(acc)

            print("Epoch: {}/{}... ".format(epoch + 1, epochs),
                  "Iter: {}".format(it),
                  "valid accuracy: {:.4f}".format(valid_acc_list[-1]))
            print("Epoch: {}/{}... ".format(epoch + 1, epochs),
                  "Iter: {}".format(it),
                  "train accuracy: {:.4f}".format(train_acc_list[-1]))

    return train_loss_list, train_acc_list, valid_acc_list

In [7]:
import torch.nn.functional as F
import os
def predict(test_data, model_file, sample_submission, input_dim, arch):
    submission_samples = pd.read_csv(sample_submission)
    submission_samples = submission_samples.set_index('id')

    data_len = len(test_data)
    print('test data len:', data_len)
    
    test_images = KaggleImageFolder('d:/DATA/dog bread/test', transform=None, mode='test').samples
    
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1)
    
    checkpoint = torch.load(model_file)
    n_classes = checkpoint['n_classes']
    model = Classifier(input_dim, n_classes)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    cuda = torch.cuda.is_available()
    if cuda:
        model.cuda()

    with torch.no_grad():
        for i, (image, x) in enumerate(zip(test_images, test_loader)):
            if cuda:
                x = x.cuda()
                torch.backends.cudnn.benchmark = True
            z = model(x)
            prob = F.softmax(z, dim=1).cpu()[0].numpy().tolist()
            image = os.path.basename(image).split('.')[0]

            submission_samples.loc[image] = prob

    submission_samples.to_csv('sample_submission_{}.csv'.format(arch))

### restnet152 0.40934

In [8]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_features['resnet152'], batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_features['resnet152'], batch_size=batch_size)
model = Classifier(train_features['resnet152'][0][0].size()[0], 120)
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-4, weight_decay=1e-5)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [9]:
# scheduler = StepLR(optimizer, step_size=100, gamma=0.1)
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(train_features['resnet152']), n_val=len(valid_features['resnet152']), 
    criterion=criterion, optimizer=optimizer, epochs=100, save_path=root+'/save_models', scheduler=None
)
model.save_model(root+'/save_models/resnet152.pth')

Epoch: 1/100... Train Loss: 5.2186
Epoch: 1/100...  Iter: 18 valid accuracy: 0.0906
Epoch: 1/100...  Iter: 18 train accuracy: 0.0922
Epoch: 2/100... Train Loss: 4.6032
Epoch: 2/100...  Iter: 35 valid accuracy: 0.2867
Epoch: 2/100...  Iter: 35 train accuracy: 0.2916
Epoch: 3/100... Train Loss: 4.3242
Epoch: 3/100...  Iter: 52 valid accuracy: 0.4126
Epoch: 3/100...  Iter: 52 train accuracy: 0.4215
Epoch: 4/100... Train Loss: 4.0269
Epoch: 4/100...  Iter: 69 valid accuracy: 0.5338
Epoch: 4/100...  Iter: 69 train accuracy: 0.5369
Epoch: 5/100... Train Loss: 3.6561
Epoch: 5/100...  Iter: 86 valid accuracy: 0.6141
Epoch: 5/100...  Iter: 86 train accuracy: 0.6215
Epoch: 6/100... Train Loss: 3.2424
Epoch: 6/100...  Iter: 103 valid accuracy: 0.6908
Epoch: 6/100...  Iter: 103 train accuracy: 0.6921
Epoch: 7/100... Train Loss: 2.8915
Epoch: 7/100...  Iter: 120 valid accuracy: 0.7540
Epoch: 7/100...  Iter: 120 train accuracy: 0.7464
Epoch: 8/100... Train Loss: 2.5281
Epoch: 8/100...  Iter: 137 val

Epoch: 61/100... Train Loss: 0.4091
Epoch: 61/100...  Iter: 1038 valid accuracy: 0.9341
Epoch: 61/100...  Iter: 1038 train accuracy: 0.9603
Epoch: 62/100... Train Loss: 0.4126
Epoch: 62/100...  Iter: 1055 valid accuracy: 0.9335
Epoch: 62/100...  Iter: 1055 train accuracy: 0.9608
Epoch: 63/100... Train Loss: 0.4085
Epoch: 63/100...  Iter: 1072 valid accuracy: 0.9362
Epoch: 63/100...  Iter: 1072 train accuracy: 0.9625
Epoch: 64/100... Train Loss: 0.3868
Epoch: 64/100...  Iter: 1089 valid accuracy: 0.9357
Epoch: 64/100...  Iter: 1089 train accuracy: 0.9653
Epoch: 65/100... Train Loss: 0.4050
Epoch: 65/100...  Iter: 1106 valid accuracy: 0.9389
Epoch: 65/100...  Iter: 1106 train accuracy: 0.9661
Epoch: 66/100... Train Loss: 0.3804
Epoch: 66/100...  Iter: 1123 valid accuracy: 0.9394
Epoch: 66/100...  Iter: 1123 train accuracy: 0.9654
Epoch: 67/100... Train Loss: 0.3765
Epoch: 67/100...  Iter: 1140 valid accuracy: 0.9378
Epoch: 67/100...  Iter: 1140 train accuracy: 0.9672
Epoch: 68/100... Tra

In [10]:
model_file = u"d:/DATA/dog bread/save_models/Classifier-checkpoint-best.pth"
sample_submission = 'sample_submission.csv'
predict(test_features['resnet152'], model_file,sample_submission,input_dim=2048, arch='resnet152' )

test data len: 10357


### denset169 0.66725

In [11]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_features['densenet169'], batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_features['densenet169'], batch_size=batch_size)
model = Classifier(train_features['densenet169'][0][0].size()[0], 120)
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-3, weight_decay=1e-5)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [12]:
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(train_features['densenet169']), n_val=len(valid_features['densenet169']), 
    criterion=criterion, optimizer=optimizer, epochs=50, save_path=root+'/save_models', scheduler=None
)
model.save_model(root+'/save_models/densenet169.pth')

Epoch: 1/50... Train Loss: 4.7359
Epoch: 1/50...  Iter: 18 valid accuracy: 0.3644
Epoch: 1/50...  Iter: 18 train accuracy: 0.3819
Epoch: 2/50... Train Loss: 3.3762
Epoch: 2/50...  Iter: 35 valid accuracy: 0.6656
Epoch: 2/50...  Iter: 35 train accuracy: 0.6633
Epoch: 3/50... Train Loss: 2.1756
Epoch: 3/50...  Iter: 52 valid accuracy: 0.7615
Epoch: 3/50...  Iter: 52 train accuracy: 0.7868
Epoch: 4/50... Train Loss: 1.5553
Epoch: 4/50...  Iter: 69 valid accuracy: 0.8167
Epoch: 4/50...  Iter: 69 train accuracy: 0.8305
Epoch: 5/50... Train Loss: 1.2303
Epoch: 5/50...  Iter: 86 valid accuracy: 0.8387
Epoch: 5/50...  Iter: 86 train accuracy: 0.8554
Epoch: 6/50... Train Loss: 1.0597
Epoch: 6/50...  Iter: 103 valid accuracy: 0.8419
Epoch: 6/50...  Iter: 103 train accuracy: 0.8713
Epoch: 7/50... Train Loss: 0.9719
Epoch: 7/50...  Iter: 120 valid accuracy: 0.8650
Epoch: 7/50...  Iter: 120 train accuracy: 0.8843
Epoch: 8/50... Train Loss: 0.8787
Epoch: 8/50...  Iter: 137 valid accuracy: 0.8687
Epo

In [13]:
model_file = u"d:/DATA/dog bread/save_models/densenet169.pth"
sample_submission = 'sample_submission.csv'
predict(test_features['densenet169'], model_file,sample_submission,input_dim=train_features['densenet169'][0][0].size()[0], arch='densenet169')

test data len: 10357


### inception_v3  3.32490

In [14]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_features['inception_v3'], batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_features['inception_v3'], batch_size=batch_size)
model = Classifier(train_features['inception_v3'][0][0].size()[0], 120)
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-3, weight_decay=1e-5)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [15]:
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(train_features['inception_v3']), n_val=len(valid_features['inception_v3']), 
    criterion=criterion, optimizer=optimizer, epochs=150, save_path=root+'/save_models', scheduler=None
)
model.save_model(root+'/save_models/inception_v3.pth')

Epoch: 1/150... Train Loss: 4.9196
Epoch: 1/150...  Iter: 18 valid accuracy: 0.0145
Epoch: 1/150...  Iter: 18 train accuracy: 0.0151
Epoch: 2/150... Train Loss: 4.7901
Epoch: 2/150...  Iter: 35 valid accuracy: 0.0118
Epoch: 2/150...  Iter: 35 train accuracy: 0.0134
Epoch: 3/150... Train Loss: 4.7794
Epoch: 3/150...  Iter: 52 valid accuracy: 0.0214
Epoch: 3/150...  Iter: 52 train accuracy: 0.0196
Epoch: 4/150... Train Loss: 4.7742
Epoch: 4/150...  Iter: 69 valid accuracy: 0.0177
Epoch: 4/150...  Iter: 69 train accuracy: 0.0151
Epoch: 5/150... Train Loss: 4.7581
Epoch: 5/150...  Iter: 86 valid accuracy: 0.0230
Epoch: 5/150...  Iter: 86 train accuracy: 0.0187
Epoch: 6/150... Train Loss: 4.7365
Epoch: 6/150...  Iter: 103 valid accuracy: 0.0402
Epoch: 6/150...  Iter: 103 train accuracy: 0.0399
Epoch: 7/150... Train Loss: 4.6874
Epoch: 7/150...  Iter: 120 valid accuracy: 0.0461
Epoch: 7/150...  Iter: 120 train accuracy: 0.0489
Epoch: 8/150... Train Loss: 4.6159
Epoch: 8/150...  Iter: 137 val

Epoch: 61/150... Train Loss: 3.7603
Epoch: 61/150...  Iter: 1038 valid accuracy: 0.1886
Epoch: 61/150...  Iter: 1038 train accuracy: 0.2148
Epoch: 62/150... Train Loss: 3.7509
Epoch: 62/150...  Iter: 1055 valid accuracy: 0.1935
Epoch: 62/150...  Iter: 1055 train accuracy: 0.2180
Epoch: 63/150... Train Loss: 3.7408
Epoch: 63/150...  Iter: 1072 valid accuracy: 0.1951
Epoch: 63/150...  Iter: 1072 train accuracy: 0.2046
Epoch: 64/150... Train Loss: 3.7247
Epoch: 64/150...  Iter: 1089 valid accuracy: 0.1768
Epoch: 64/150...  Iter: 1089 train accuracy: 0.2033
Epoch: 65/150... Train Loss: 3.7464
Epoch: 65/150...  Iter: 1106 valid accuracy: 0.1935
Epoch: 65/150...  Iter: 1106 train accuracy: 0.2139
Epoch: 66/150... Train Loss: 3.7533
Epoch: 66/150...  Iter: 1123 valid accuracy: 0.1811
Epoch: 66/150...  Iter: 1123 train accuracy: 0.1993
Epoch: 67/150... Train Loss: 3.7549
Epoch: 67/150...  Iter: 1140 valid accuracy: 0.1811
Epoch: 67/150...  Iter: 1140 train accuracy: 0.2054
Epoch: 68/150... Tra

Epoch: 120/150... Train Loss: 3.5784
Epoch: 120/150...  Iter: 2041 valid accuracy: 0.2036
Epoch: 120/150...  Iter: 2041 train accuracy: 0.2405
Epoch: 121/150... Train Loss: 3.5605
Epoch: 121/150...  Iter: 2058 valid accuracy: 0.2170
Epoch: 121/150...  Iter: 2058 train accuracy: 0.2633
Epoch: 122/150... Train Loss: 3.5287
Epoch: 122/150...  Iter: 2075 valid accuracy: 0.2208
Epoch: 122/150...  Iter: 2075 train accuracy: 0.2563
Epoch: 123/150... Train Loss: 3.5249
Epoch: 123/150...  Iter: 2092 valid accuracy: 0.2170
Epoch: 123/150...  Iter: 2092 train accuracy: 0.2611
Epoch: 124/150... Train Loss: 3.5373
Epoch: 124/150...  Iter: 2109 valid accuracy: 0.2101
Epoch: 124/150...  Iter: 2109 train accuracy: 0.2665
Epoch: 125/150... Train Loss: 3.5540
Epoch: 125/150...  Iter: 2126 valid accuracy: 0.2138
Epoch: 125/150...  Iter: 2126 train accuracy: 0.2597
Epoch: 126/150... Train Loss: 3.5377
Epoch: 126/150...  Iter: 2143 valid accuracy: 0.1967
Epoch: 126/150...  Iter: 2143 train accuracy: 0.2541

In [16]:
model_file = u"d:/DATA/dog bread/save_models/inception_v3.pth"
sample_submission = 'sample_submission.csv'
predict(test_features['inception_v3'], model_file,sample_submission,input_dim=train_features['inception_v3'][0][0].size()[0], arch='inception_v3')

test data len: 10357


### cat 2 features 0.40715

In [17]:
def get_mix_features_labels(features):
    mix_features = []
    for x1, x2 in zip(features['resnet152'], features['densenet169']):
        assert x1[1] == x2[1], "labels did not match"
        mix_features.append((torch.cat((x1[0], x2[0]), 0), x1[1]))
    return mix_features

In [18]:
def get_mix_features(features):
    mix_features = []
    for x1, x2 in zip(features['resnet152'], features['densenet169']):
        mix_features.append(torch.cat((x1, x2,), 0))
    return mix_features

In [19]:
mix_trian_features = get_mix_features_labels(train_features)
mix_valid_features = get_mix_features_labels(valid_features)
mix_test_features = get_mix_features(test_features)

In [20]:
mix_trian_features[2][0].size()

torch.Size([3712])

In [21]:
mix_valid_features[0][0].size()

torch.Size([3712])

In [22]:
mix_test_features[0].size()

torch.Size([3712])

In [23]:
input_dim = mix_trian_features[0][0].size()[0]

In [24]:
train_dataloader = torch.utils.data.DataLoader(dataset=mix_trian_features, batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=mix_valid_features, batch_size=batch_size)
model = Classifier(input_dim, 120)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [25]:
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-4, weight_decay=1e-5)
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(mix_trian_features), n_val=len(mix_valid_features), 
    criterion=criterion, optimizer=optimizer, epochs=100, save_path=root+'/save_models', scheduler=None
)

Epoch: 1/100... Train Loss: 5.1737
Epoch: 1/100...  Iter: 18 valid accuracy: 0.1131
Epoch: 1/100...  Iter: 18 train accuracy: 0.1108
Epoch: 2/100... Train Loss: 4.5534
Epoch: 2/100...  Iter: 35 valid accuracy: 0.3151
Epoch: 2/100...  Iter: 35 train accuracy: 0.3058
Epoch: 3/100... Train Loss: 4.2735
Epoch: 3/100...  Iter: 52 valid accuracy: 0.4491
Epoch: 3/100...  Iter: 52 train accuracy: 0.4586
Epoch: 4/100... Train Loss: 3.8826
Epoch: 4/100...  Iter: 69 valid accuracy: 0.5552
Epoch: 4/100...  Iter: 69 train accuracy: 0.5646
Epoch: 5/100... Train Loss: 3.4460
Epoch: 5/100...  Iter: 86 valid accuracy: 0.6292
Epoch: 5/100...  Iter: 86 train accuracy: 0.6378
Epoch: 6/100... Train Loss: 3.0172
Epoch: 6/100...  Iter: 103 valid accuracy: 0.6790
Epoch: 6/100...  Iter: 103 train accuracy: 0.7005
Epoch: 7/100... Train Loss: 2.6117
Epoch: 7/100...  Iter: 120 valid accuracy: 0.7497
Epoch: 7/100...  Iter: 120 train accuracy: 0.7573
Epoch: 8/100... Train Loss: 2.2454
Epoch: 8/100...  Iter: 137 val

Epoch: 61/100... Train Loss: 0.3214
Epoch: 61/100...  Iter: 1038 valid accuracy: 0.9475
Epoch: 61/100...  Iter: 1038 train accuracy: 0.9786
Epoch: 62/100... Train Loss: 0.3106
Epoch: 62/100...  Iter: 1055 valid accuracy: 0.9475
Epoch: 62/100...  Iter: 1055 train accuracy: 0.9795
Epoch: 63/100... Train Loss: 0.3053
Epoch: 63/100...  Iter: 1072 valid accuracy: 0.9464
Epoch: 63/100...  Iter: 1072 train accuracy: 0.9804
Epoch: 64/100... Train Loss: 0.2989
Epoch: 64/100...  Iter: 1089 valid accuracy: 0.9544
Epoch: 64/100...  Iter: 1089 train accuracy: 0.9823
Epoch: 65/100... Train Loss: 0.2780
Epoch: 65/100...  Iter: 1106 valid accuracy: 0.9475
Epoch: 65/100...  Iter: 1106 train accuracy: 0.9813
Epoch: 66/100... Train Loss: 0.2865
Epoch: 66/100...  Iter: 1123 valid accuracy: 0.9523
Epoch: 66/100...  Iter: 1123 train accuracy: 0.9836
Epoch: 67/100... Train Loss: 0.2781
Epoch: 67/100...  Iter: 1140 valid accuracy: 0.9491
Epoch: 67/100...  Iter: 1140 train accuracy: 0.9844
Epoch: 68/100... Tra

In [26]:
# optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
#                                  lr=1e-4, weight_decay=1e-5)
# scheduler = StepLR(optimizer, step_size=100, gamma=0.1)
# train_loss_list, train_acc_list, valid_acc_list = train(
#     model, train_dataloader, valid_dataloader, n_train=len(mix_trian_features), n_val=len(mix_valid_features), 
#     criterion=criterion, optimizer=optimizer, epochs=50, save_path=root+'/save_models', scheduler=scheduler
# )

In [27]:
model.save_model(root+'/save_models/mix-model.pth')

In [28]:
model_file = u"d:/DATA/dog bread/save_models/mix-model.pth"
sample_submission = 'sample_submission.csv'
predict(mix_test_features, model_file,sample_submission,input_dim=input_dim, arch='mix')

test data len: 10357
